In [1]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [2]:
df = ssql("""
SELECT t1.date,
       t1.hour,
       count(t1.distinct_id)AS `DNU`,
       count(t2.distinct_id) AS `login_r1`,
       count(t2.distinct_id)/count(t1.distinct_id) AS `app_r1_rate`,
       count(t3.distinct_id) AS `game_r1`,
       count(t3.distinct_id)/count(t1.distinct_id)AS `game_r1_rate`
FROM
  (SELECT date, hour(time)AS hour,
                distinct_id
   FROM events
   WHERE event = 'register'
     AND appId = '20014'
     AND date BETWEEN current_date() - interval 30 DAY AND current_date()
   GROUP BY 1,
            2,
            3)t1
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'login'
     AND appId = '20014'
     AND date BETWEEN current_date() - interval 30 DAY AND current_date()
   GROUP BY 1,
            2)t2 ON datediff(t2.date,t1.date) = 2
AND t1.distinct_id = t2.distinct_id
LEFT JOIN
  (SELECT date,distinct_id
   FROM events
   WHERE event = 'gameStart'
     AND gameTypeId = 1800
     AND date BETWEEN current_date() - interval 30 DAY AND current_date()
   GROUP BY 1,
            2)t3 ON datediff(t3.date,t1.date) = 2
AND t1.distinct_id = t3.distinct_id
GROUP BY 1,
         2
ORDER BY 1,
         2""")

In [3]:
df['date'] = pd.to_datetime(df['date'])

In [4]:
df

,date,hour,login_r1,dnu,game_r1_rate,game_r1,app_r1_rate
0,2020-10-14,12,0,1,0.0,0,0.0
1,2020-10-14,13,0,1,0.0,0,0.0
2,2020-10-14,15,0,1,0.0,0,0.0
3,2020-10-14,16,0,4,0.0,0,0.0
4,2020-10-14,17,0,4,0.0,0,0.0
...,...,...,...,...,...,...,...
691,2020-11-13,6,0,459,0.0,0,0.0
692,2020-11-13,7,0,603,0.0,0,0.0
693,2020-11-13,8,0,592,0.0,0,0.0
694,2020-11-13,9,0,757,0.0,0,0.0


In [5]:
df = df[df['date']<='2020-11-11']
df

,date,hour,login_r1,dnu,game_r1_rate,game_r1,app_r1_rate
0,2020-10-14,12,0,1,0.000,0,0.000
1,2020-10-14,13,0,1,0.000,0,0.000
2,2020-10-14,15,0,1,0.000,0,0.000
3,2020-10-14,16,0,4,0.000,0,0.000
4,2020-10-14,17,0,4,0.000,0,0.000
...,...,...,...,...,...,...,...
656,2020-11-11,19,171,4546,0.033,149,0.038
657,2020-11-11,20,229,4888,0.038,188,0.047
658,2020-11-11,21,216,4177,0.045,189,0.052
659,2020-11-11,22,256,3461,0.072,249,0.074


In [6]:
df_1 = df.query("week == 6 or week == 7")
df_1

UndefinedVariableError: name 'week' is not defined

In [60]:
cond = df['week'].isin(['1','2','3','4','5'])
df_2 = df[cond]
df_2

,date,hour,login_r1,dnu,game_r1_rate,game_r1,app_r1_rate,week
121,2020-10-20,0,6,30,0.200,6,0.200,2
122,2020-10-20,1,10,25,0.360,9,0.400,2
123,2020-10-20,2,5,7,0.714,5,0.714,2
124,2020-10-20,3,5,10,0.500,5,0.500,2
125,2020-10-20,4,3,5,0.400,2,0.600,2
...,...,...,...,...,...,...,...,...
620,2020-11-09,19,874,14862,0.051,758,0.059,1
621,2020-11-09,20,943,16097,0.052,839,0.059,1
622,2020-11-09,21,919,12242,0.066,806,0.075,1
623,2020-11-09,22,801,8935,0.083,743,0.090,1


In [61]:
df_2.week.unique()

array([2, 3, 4, 5, 1])

In [62]:
df_1 = df_1.groupby(['hour'])['game_r1_rate','app_r1_rate'].mean()
df_1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,game_r1_rate,app_r1_rate
hour,,
0,0.170500,0.201667
1,0.193667,0.224667
2,0.212667,0.246667
3,0.203500,0.246833
4,0.231500,0.261667
5,0.192000,0.213167
6,0.110167,0.127167
7,0.109833,0.133500
8,0.109333,0.130333


In [63]:
df_1.describe()

,game_r1_rate,app_r1_rate
count,24.000000,24.000000
mean,0.157340,0.185813
std,0.043870,0.049105
min,0.104167,0.126667
25%,0.117292,0.139875
50%,0.151000,0.181333
75%,0.192417,0.223917
max,0.245833,0.284000


In [64]:
df_2 = df_2.groupby(['hour'])['game_r1_rate','app_r1_rate'].mean()
df_2

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,game_r1_rate,app_r1_rate
hour,,
0,0.256600,0.294533
1,0.273733,0.317800
2,0.322667,0.361667
3,0.315800,0.350533
4,0.302000,0.354067
5,0.274400,0.306000
6,0.279400,0.318400
7,0.244800,0.287533
8,0.243133,0.302133


In [65]:
df_2.describe()

,game_r1_rate,app_r1_rate
count,24.000000,24.000000
mean,0.268700,0.311292
std,0.022173,0.021053
min,0.243133,0.287533
25%,0.254183,0.297150
50%,0.260900,0.304233
75%,0.274617,0.317800
max,0.322667,0.361667


In [66]:
df_realName = ssql("""SELECT date,distinct_id,
            appId,
            real_name_type,
            idNum
FROM events
WHERE event = 'regRealName'
  AND date BETWEEN '2020-10-23' AND current_date()
  AND appId IN ('20014','30015')
GROUP BY 1,
         2,
         3,
         4,5""")
df_realName

,date,real_name_type,distinct_id,appid,idnum
0,2020-11-07,id_card,143588885,20014,130434200503147313
1,2020-10-27,phone,115607298,20014,
2,2020-11-03,id_card,112415572,20014,211481200203254710
3,2020-11-05,phone,115979278,20014,
4,2020-11-08,phone,131369358,20014,
...,...,...,...,...,...
988878,2020-11-07,phone,142935449,20014,
988879,2020-11-07,phone,120554089,20014,
988880,2020-11-07,phone,128824374,20014,
988881,2020-11-10,id_card,19100830,20014,131125200207221425


In [186]:
df_ios = df_realName[df_realName['appid'] == '30015']
df_ios

,date,real_name_type,distinct_id,appid,idnum
11,2020-11-06,id_card,140162743,30015,220281200509265412
39,2020-11-07,id_card,139069285,30015,410721198906160530
69,2020-11-12,id_card,120131313,30015,622223199505206113
72,2020-11-06,id_card,134241855,30015,341004198106170819
79,2020-11-08,id_card,142363322,30015,210105199612181410
...,...,...,...,...,...
988791,2020-10-26,id_card,148834766,30015,362421199812111733
988802,2020-10-27,id_card,136012977,30015,350321198504090738
988810,2020-10-30,id_card,117485862,30015,210422199509090011
988833,2020-10-31,id_card,122016534,30015,330402197508152113


In [187]:
df_ios = df_ios.reset_index(drop=True)
df_ios

,date,real_name_type,distinct_id,appid,idnum
0,2020-11-06,id_card,140162743,30015,220281200509265412
1,2020-11-07,id_card,139069285,30015,410721198906160530
2,2020-11-12,id_card,120131313,30015,622223199505206113
3,2020-11-06,id_card,134241855,30015,341004198106170819
4,2020-11-08,id_card,142363322,30015,210105199612181410
...,...,...,...,...,...
41485,2020-10-26,id_card,148834766,30015,362421199812111733
41486,2020-10-27,id_card,136012977,30015,350321198504090738
41487,2020-10-30,id_card,117485862,30015,210422199509090011
41488,2020-10-31,id_card,122016534,30015,330402197508152113


In [188]:
def proce(x):
    if x:
        y = 2020 - int(x[6:10])
        if y < 10:
            return '小于10岁'
        elif y < 15:
            return '10-15岁'
        elif y < 18:
            return '15-18岁'
        elif y < 25:
            return '18-25岁'
        elif y < 35:
            return '25-35岁'
        else:
            return '35岁以上'
    else:
        return '未获取到年龄'

In [189]:
df_ios['bir'] = df_ios['idnum'].apply(proce)
df_ios

,date,real_name_type,distinct_id,appid,idnum,bir
0,2020-11-06,id_card,140162743,30015,220281200509265412,15-18岁
1,2020-11-07,id_card,139069285,30015,410721198906160530,25-35岁
2,2020-11-12,id_card,120131313,30015,622223199505206113,25-35岁
3,2020-11-06,id_card,134241855,30015,341004198106170819,35岁以上
4,2020-11-08,id_card,142363322,30015,210105199612181410,18-25岁
...,...,...,...,...,...,...
41485,2020-10-26,id_card,148834766,30015,362421199812111733,18-25岁
41486,2020-10-27,id_card,136012977,30015,350321198504090738,35岁以上
41487,2020-10-30,id_card,117485862,30015,210422199509090011,25-35岁
41488,2020-10-31,id_card,122016534,30015,330402197508152113,35岁以上


In [190]:
df_ios['date'] = pd.to_datetime(df_ios['date'])
df_ios

,date,real_name_type,distinct_id,appid,idnum,bir
0,2020-11-06,id_card,140162743,30015,220281200509265412,15-18岁
1,2020-11-07,id_card,139069285,30015,410721198906160530,25-35岁
2,2020-11-12,id_card,120131313,30015,622223199505206113,25-35岁
3,2020-11-06,id_card,134241855,30015,341004198106170819,35岁以上
4,2020-11-08,id_card,142363322,30015,210105199612181410,18-25岁
...,...,...,...,...,...,...
41485,2020-10-26,id_card,148834766,30015,362421199812111733,18-25岁
41486,2020-10-27,id_card,136012977,30015,350321198504090738,35岁以上
41487,2020-10-30,id_card,117485862,30015,210422199509090011,25-35岁
41488,2020-10-31,id_card,122016534,30015,330402197508152113,35岁以上


In [193]:
a = df_ios.loc[0,'date']

In [194]:
a

Timestamp('2020-11-06 00:00:00')

In [200]:
a.weekday()+1

5

In [171]:
df_ios.bir.unique()

array(['未成年', '成年', '未获取到年龄'], dtype=object)

In [180]:
df_ios = df_ios.groupby(['bir']).count()
df_ios

,date,real_name_type,distinct_id,appid,idnum
bir,,,,,
10-15岁,1869,1869,1869,1869,1869
15-18岁,1163,1163,1163,1163,1163
18-25岁,12597,12597,12597,12597,12597
25-35岁,6879,6879,6879,6879,6879
35岁以上,5572,5572,5572,5572,5572
小于10岁,278,278,278,278,278
未获取到年龄,13132,13132,13132,13132,13132


In [183]:
df_ios.to_clipboard()

In [78]:
df_ios = pd.DataFrame(df_ios.groupby(['real_name_type'])['distinct_id'].count())
df_ios

,distinct_id
real_name_type,
id_card,28358
phone,13132


In [82]:
df_ios.index

Index(['id_card', 'phone'], dtype='object', name='real_name_type')

In [69]:
df_android = df_realName[df_realName['appid'] == '20014']
df_android

,date,real_name_type,distinct_id,appid,idnum
0,2020-11-07,id_card,143588885,20014,130434200503147313
1,2020-10-27,phone,115607298,20014,
2,2020-11-03,id_card,112415572,20014,211481200203254710
3,2020-11-05,phone,115979278,20014,
4,2020-11-08,phone,131369358,20014,
...,...,...,...,...,...
988878,2020-11-07,phone,142935449,20014,
988879,2020-11-07,phone,120554089,20014,
988880,2020-11-07,phone,128824374,20014,
988881,2020-11-10,id_card,19100830,20014,131125200207221425


In [176]:
df_android['bir'] = df_android['idnum'].apply(proce)
df_android

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,real_name_type,distinct_id,appid,idnum,bir
0,2020-11-07,id_card,143588885,20014,130434200503147313,15-18岁
1,2020-10-27,phone,115607298,20014,,未获取到年龄
2,2020-11-03,id_card,112415572,20014,211481200203254710,18-25岁
3,2020-11-05,phone,115979278,20014,,未获取到年龄
4,2020-11-08,phone,131369358,20014,,未获取到年龄
...,...,...,...,...,...,...
988878,2020-11-07,phone,142935449,20014,,未获取到年龄
988879,2020-11-07,phone,120554089,20014,,未获取到年龄
988880,2020-11-07,phone,128824374,20014,,未获取到年龄
988881,2020-11-10,id_card,19100830,20014,131125200207221425,18-25岁


In [184]:
df_android = df_android.groupby(['bir']).count()
df_android

,date,real_name_type,distinct_id,appid,idnum
bir,,,,,
10-15岁,47913,47913,47913,47913,47913
15-18岁,23347,23347,23347,23347,23347
18-25岁,68531,68531,68531,68531,68531
25-35岁,27953,27953,27953,27953,27953
35岁以上,110332,110332,110332,110332,110332
小于10岁,5466,5466,5466,5466,5466
未获取到年龄,663851,663851,663851,663851,663851


In [185]:
df_android.to_clipboard()